In [1]:
import os
import json
import gzip
import pandas as pd
from urllib.request import urlopen
from itertools import combinations
import re
import torch
import numpy as np
import pickle

In [44]:
df_a = pd.read_csv("Datasets/Structured/Walmart-Amazon/tableA.csv")
df_b = pd.read_csv("Datasets/Structured/Walmart-Amazon/tableB.csv")
train0 = pd.read_csv("Datasets/Structured/Walmart-Amazon/train_0.csv")
valid0 = pd.read_csv("Datasets/Structured/Walmart-Amazon/valid_0.csv")
test0 = pd.read_csv("Datasets/Structured/Walmart-Amazon/test_0.csv")

In [45]:
categories_dict = dict()

# Electronics Specific Catgories

In [47]:
cameras_category = {'camcorders', 'camera and camcorder accessories', 'digital cameras', 'dvd accessories', 'dvd players', 
                   'electronics - general', 'film cameras','bullet cameras', 'camcorder batteries', 
                   'camcorders', 'camera batteries', 'camera lenses', 'camera photo', 'digital camera accessories', 
                    'digital cameras', 'digital slr camera bundles', 'digital slr cameras', 'filters', 'film','flashes', 
                    'floppy diskettes', 'hidden cameras', 'lens accessories', 'photo studio', 'photo video design', 
                   'point shoot digital cameras','underwater photography', 'webcams', 'binoculars','game cameras',
                   'tripods', 'photography - general'}

for cat in cameras_category:
    if cat in categories_dict.keys():
        categories_dict[cat]['general'].add('general_Electronics')
        categories_dict[cat]['main'].add('cameras_category')
    else:
        categories_dict[cat] = {'general': {'general_Electronics'}, 'main': {'cameras_category'}}

In [48]:
phones_category = {'phone accessories', 'phones','prepaid wireless phones', 'walkie-talkie/frs',
                  'answering devices', 'caller id displays', 'corded telephones', 'corded-cordless combo telephones', 
                   'cordless telephones', 'frs two-way radios', 'gmrs-frs two-way radios',
                  'telephone accessories', 'telephones accessories', 'voip', 'answering devices', }

for cat in phones_category:
    if cat in categories_dict.keys():
        categories_dict[cat]['general'].add('general_Electronics')
        categories_dict[cat]['main'].add('phones_category')
    else:
        categories_dict[cat] = {'general': {'general_Electronics'}, 'main': {'phones_category'}}

In [49]:
smartphones_category = {'post-paid cellphones','desktop chargers', 'faceplates', 'fan shop', 'no-contract phones',
                       'phones with plans','touch pads', 'touch screen tablet accessories','unlocked phones',
                       'bundles', 'in-dash navigation'}

for cat in smartphones_category:
    if cat in categories_dict.keys():
        categories_dict[cat]['general'].add('general_Electronics')
        categories_dict[cat]['main'].add('smartphones_category')
    else:
        categories_dict[cat] = {'general': {'general_Electronics'}, 'main': {'smartphones_category'}}

In [50]:
audio_category = {'stereos/audio', 'ipod','mp3', 'mp3 accessories','portable audio',
                  'audio cables', 'audio conferencing', 'audio video accessories', 
                  'audio-video kits', 'audio-video shelving','bluetooth headsets', 'cd players', 'cd-mp3 players', 
                 'component subwoofers', 'digital media devices','home audio crossovers', 'home audio theater', 
                 'home theater systems', 'mp3 player accessories', 'mp3 players', 'portable audio video',
                 'tweeter diaphragms','boomboxes', 'audio video'}
                  
for cat in audio_category:
    if cat in categories_dict.keys():
        categories_dict[cat]['general'].add('general_Electronics')
        categories_dict[cat]['main'].add('audio_category')
    else:
        categories_dict[cat] = {'general': {'general_Electronics'}, 'main': {'audio_category'}}

In [51]:
computers_category = {'computers', 'electronics - general', 'laptop computers','software', 'business office',
                     'computer accessories', 'computer components','computer monitor mounts', 'computer speakers', 
                      'computers accessories', 'desktops','document creation','flowcharts','game cameras', 'gaming mice',
                     'laptops', 'netbooks', 'photo editing', 'professional design', 'programmable','software',
                     'tablets','webcams', 'controllers', 'hard drives'}

for cat in computers_category:
    if cat in categories_dict.keys():
        categories_dict[cat]['general'].add('general_Electronics')
        categories_dict[cat]['main'].add('computers_category')
    else:
        categories_dict[cat] = {'general': {'general_Electronics'}, 'main': {'computers_category'}}

In [52]:
tv_and_dvd_category = {'electronics - general', 'electronics : flat panel tv','television', 'tv accessories', 
                      'video receiving & installation','digital media devices','dvd home theater', 'dvd players', 
                       'dvd r discs', 'dvd rw discs', 'dvd-r discs', 'dvd-ram discs', 'dvi cables', 'film', 'overhead video',
                       'portable audio video', 'portable dvd players', 'portable novelty tvs', 
                       'professional video accessories', 'television stands entertainment centers', 
                       'television video', 'televisions', 'televisions video','tv accessories', 
                       'tv mounts', 'tv-dvd combinations', 'tvs', 'home theater'}

for cat in tv_and_dvd_category:
    if cat in categories_dict.keys():
        categories_dict[cat]['general'].add('general_Electronics')
        categories_dict[cat]['main'].add('tv_and_dvd_category')
    else:
        categories_dict[cat] = {'general': {'general_Electronics'}, 'main': {'tv_and_dvd_category'}}

In [53]:
hardware_and_external_accessories_category = {'electronics - general','memory','mice', 'monitors', 'networking', 'printers', 
                                              'satellite radio','storage', 'usb drives', 'access-control keypads', 'accessories',
                                              'accessories supplies', 'accessory kits', 'adapter rings', 'adapters', 'cable security devices',
                                             'amplifier installation', 'amplifier wiring kits', 'amplifiers', 'amps','cables interconnects', 
                                             'analog-to-digital dtv converters','auxiliary input adapters', 'bar code scanners',
                                             'basic', 'bd-r discs', 'bd-re discs', 'blank media', 'blu-ray disc players', 
                                              'blu-ray drives','business paper products','calculator accessories', 'calculators',
                                             'cb radios','cd-r discs', 'cd-rw discs', 'changers', 'chargers', 'chargers adapters', 
                                              'chargers cables', 'coaxial speakers','component subwoofers', 'component video', 
                                              'computer cable adapters', 'computer monitor mounts', 'computer speakers', 
                                              'computers accessories', 'connectors adapters', 'data cables', 'data cartridges', 
                                              'desktops','digital media devices', 'disc players recorders','distribution',
                                             'distribution panels', 'dlt cleaning cartridges', 'docking stations', 'dome cameras', 
                                             'electrical', 'electronics', 'equalizers', 'ethernet cables', 'external data storage', 
                                              'external floppy drives', 'external hard drives', 'faceplates','filters',
                                             'firewire cables', 'firewire port cards', 'graphics cards', 'graphics tablets', 
                                             'hard drive enclosures', 'hardware', 'hdmi cables', 'headphone accessories', 
                                              'headphones', 'headsets', 'headsets microphones', 'hubs', 'i o port cards', 
                                              'inkjet all-in-one printers', 'inkjet printer ink', 
                                              'inkjet printer paper', 'internal dvd drives', 'internal hard drives', 
                                              'internal optical drives', 'internal sound cards','keyboard mouse combos', 
                                              'keyboards', 'keyboards mice input devices', 'keyboards styluses', 'kvm switches', 
                                              'laptop netbook computer accessories','laser all-in-one printers', 
                                              'laser pointers', 'laser printer toner', 'laser printers', 'media storage organization', 'memo scratch pads', 
                                              'memory', 'memory card adapters', 'memory card readers', 'memory cards', 
                                              'microphones accessories', 'modems', 'monitor accessories', 'monitor arms stands', 'monitors', 
                                              'motherboards', 'mp3 player cables adapters', 'network adapters', 'network attached storage', 
                                              'network cards', 'networking products', 'office electronics','office electronics accessories', 
                                             'optical drives', 'pda handheld accessories','photo background support equipment', 
                                             'plug play satellite radios', 'point shoot digital camera bundles', 
                                              'portable cd players', 'postal scales', 'power adapters', 'power amplifiers', 'power cables', 
                                              'power conditioners', 'power converters', 'power ground cable', 'power inverters', 'power packs', 
                                              'power strips', 'power supplies', 'power-cable terminals', 'powerline network adapters', 'preamps', 
                                              'presentation pointers', 'presentation remotes', 'presentation supplies', 'print servers', 'printer accessories', 
                                              'printer control cards', 'printer drawers', 'printer feeders', 'printer finishers', 'printer ink toner', 
                                              'printer labels laser inkjet', 'printer photoconductors', 'printer roll holders', 'printer staplers', 
                                              'printer staples', 'printer transfer belts', 'computer cases'
                                              'printer transfer rollers', 'printer transfer units', 'printer trays', 
                                              'printers', 'printers scanners', 'printing', 'projection screens', 'projector accessories', 
                                              'projector bags cases', 'projector mounts', 'radar detectors', 'radio antennas', 'radio scanners', 
                                              'radios', 'radios mp3 cd players', 'rca cables', 'remote controls', 'remote-control extenders',
                                              'routers', 's-video cables', 'safety', 'satellite radio', 'satellite radio tuners adapters', 
                                              'scanner accessories', 'scanners', 'scientific', 'scsi cables', 'scsi port cards', 
                                              'security monitors displays', 'serial cables', 'shortwave', 'signal boosters', 
                                              'speaker boxes', 'speaker cables', 'speaker connectors', 'speaker installation','speaker mounts',
                                              'speaker parts components', 'speaker stands', 'speaker systems', 'speaker wire', 'speakers', 
                                              'sports outdoor gps', 'spotting scopes', 'stage studio cables', 'stereo amplifiers', 'stereos', 
                                              'storage file boxes', 'storage presentation materials', 'studio environment', 'styli',
                                              'subwoofer amplifiers', 'subwoofer boxes and enclosures', 'subwoofer cabinets',
                                              'subwoofer cables', 'subwoofers', 'surge protectors', 'switchers', 'switches', 'synch pc cords', 
                                             'tape decks', 'time cards', 'time clocks', 'timers', 'trackballs', 'turntables','tweeter diaphragms',
                                             'two-way radios accessories', 'typewriters', 'underwater lighting', 
                                              'uninterrupted power supply ups', 'universal fm cassette adapters','upconverting dvd players', 
                                              'usb cables', 'usb flash drives', 'usb network adapters', 'usb port cards', 'vhs', 'video cables', 
                                              'video capture cards', 'video converters', 'video glasses', 'voip', 
                                              'video multiplexers quads', 'video projector accessories', 'video projectors', 'video transmission systems', 
                                             'wire management', 'wired headsets', 'wireless access points', 'wireless microphones', 'wires', 
                                              'lamps', 'lighting', 'wiring harnesses', 'office furniture lighting', 'outlet plates',
                                             'wall chargers', 'wall plates connectors', 'controllers','mounts', 'hard drives',
                                             'computers', 'bags cases', 'stationery & office machinery', 'cases', 
                                              'coin button cell', 'screen protectors', 'computer cases', 'cases bags',
                                             'ac adapters', 'antennas', 'electronics-inflexible kit', 'all-in-one printers',
                                             'printer transfer rollers'}

for cat in hardware_and_external_accessories_category:
    if cat in categories_dict.keys():
        categories_dict[cat]['general'].add('general_Electronics')
        categories_dict[cat]['main'].add('hardware_and_external_accessories_category')
    else:
        categories_dict[cat] = {'general': {'general_Electronics'}, 'main': {'hardware_and_external_accessories_category'}}

In [54]:
navigation_category = {'handheld gps', 'gps system accessories', 'handheld gps', 
                       'handheld portable satellite radios', 'hiking gps units'}

for cat in navigation_category:
    if cat in categories_dict.keys():
        categories_dict[cat]['general'].add('general_Electronics')
        categories_dict[cat]['main'].add('navigation_category')
    else:
        categories_dict[cat] = {'general': {'general_Electronics'}, 'main': {'navigation_category'}}

In [55]:
batteries_category = {'12v', '6v', '9v', 'a', 'aa', 'aaa', 'batteries', 'batteries accessories', 'batteries chargers', 
                      'batteries chargers accessories', 'battery chargers', 'battery wiring terminals','c', 
                     'camcorder batteries', 'camera batteries', 'd', 'household batteries'}

for cat in batteries_category:
    if cat in categories_dict.keys():
        categories_dict[cat]['general'].add('general_Electronics')
        categories_dict[cat]['main'].add('batteries_category')
    else:
        categories_dict[cat] = {'general': {'general_Electronics'}, 'main': {'batteries_category'}}

# Personal Equipment

In [56]:
art_and_work_category = {'business cards',  'colored paper','coin button cell', 'complete tripod units', 
                           'continuous-form labels', 'covers', 'covers skins', 'file folder labels', 
                          'impact dot matrix printer ribbons', 'label makers', 'labeling tapes', 'labels stickers',
                           'laminating supplies', 'view binders', 'wide-format paper','wrist rests', 
                        'screen protectors', 'self-stick note pad holders', 'self-stick notes', 'shipping labels',
                        'portfolios', 'postcards', 'roll paper', 'screen filters', 'screen protector foils',
                        'mouse pads', 'binocular accessories', 'photo paper', 'other office equipment',
                        'looseleaf binder paper', 'case fans', 'disc jewel cases', 'disc storage wallets', 
                        'paper', 'selector boxes', 'accessories apparel', 'accessories apparel', 'binding machines',
                         'personal care', 'index dividers'}

for cat in art_and_work_category:
    if cat in categories_dict.keys():
        categories_dict[cat]['general'].add('general_personal')
        categories_dict[cat]['main'].add('art_and_work_category')
    else:
        categories_dict[cat] = {'general': {'general_personal'}, 'main': {'art_and_work_category'}}

In [57]:
hobbies_category  = {'boating gps units chartplotters', 'fishfinders', 'game cameras',
                    'hiking gps units', 'binocular accessories', 'english dictionaries', 'tripods',
                    'fishing and boating', 'fish finder', 'toys - games'}

for cat in hobbies_category:
    if cat in categories_dict.keys():
        categories_dict[cat]['general'].add('general_personal')
        categories_dict[cat]['main'].add('hobbies_category')
    else:
        categories_dict[cat] = {'general': {'general_personal'}, 'main': {'hobbies_category'}}

In [58]:
bags_and_cases_category = {'bags cases', 'cases','disc storage wallets','faceplates', 'fan shop',
                          'laptop netbook computer accessories','badge holders','cases sleeves',
                          'hard drive bags', 'cases bags', 'handbags', 'computer cases'}

for cat in bags_and_cases_category:
    if cat in categories_dict.keys():
        categories_dict[cat]['general'].add('general_personal')
        categories_dict[cat]['main'].add('bags_and_cases_category')
    else:
        categories_dict[cat] = {'general': {'general_personal'}, 'main': {'bags_and_cases_category'}}

In [59]:
dress_category = {'armbands', 'binocular accessories', 'personal care'}

for cat in dress_category:
    if cat in categories_dict.keys():
        categories_dict[cat]['general'].add('general_personal')
        categories_dict[cat]['main'].add('dress_category')
    else:
        categories_dict[cat] = {'general': {'general_personal'}, 'main': {'dress_category'}}

# House

In [60]:
kitchen_category = {'house wares'}

for cat in kitchen_category:
    if cat in categories_dict.keys():
        categories_dict[cat]['general'].add('general_house')
        categories_dict[cat]['main'].add('kitchen_category')
    else:
        categories_dict[cat] = {'general': {'general_house'}, 'main': {'kitchen_category'}}

In [61]:
home_care_category = {'home care', 'cleaning repair', 'cooling pads', 'lamps', 'lighting', 
                     'selector boxes', 'garden - general', 'surveillance cameras'}

for cat in home_care_category:
    if cat in categories_dict.keys():
        categories_dict[cat]['general'].add('general_house')
        categories_dict[cat]['main'].add('home_care_category')
    else:
        categories_dict[cat] = {'general': {'general_house'}, 'main': {'home_care_category'}}

In [62]:
office_category = {'address labels', 'all-purpose labels', 'binder index dividers', 
                   'binding machine supplies',  'other office equipment', 'photo video design','paper',  
                   'shredders',  'stands', 'tripods', 'stationery & office machinery', 'cleaning repair','selector boxes',
                  'cooling pads', 'desktop staplers','hanging folders interior folders','office furniture lighting',
                  'stands', 'laminating supplies', 'folder labels', 'binding machines', 'all-in-one printers', 'index dividers'}

for cat in office_category:
    if cat in categories_dict.keys():
        categories_dict[cat]['general'].add('general_house')
        categories_dict[cat]['main'].add('office_category')
    else:
        categories_dict[cat] = {'general': {'general_house'}, 'main': {'office_category'}}

In [63]:
furnitures_category = {'furniture', 'home care', 'rugs', 'cleaning repair', 'cooling pads', 'footrests', 
                      'office furniture lighting', 'garden - general'}

for cat in furnitures_category:
    if cat in categories_dict.keys():
        categories_dict[cat]['general'].add('general_house')
        categories_dict[cat]['main'].add('furnitures_category')
    else:
        categories_dict[cat] = {'general': {'general_house'}, 'main': {'furnitures_category'}}

In [64]:
security_category = {'antitheft', 'complete surveillance systems', 'digital security recorders','dome cameras',
                    'hidden cameras', 'home security systems', 'household sensors alarms', 
                    'security sensors alarms', 'security surveillance', 'simulated cameras',
                    'surveillance cameras', 'surveillance video recorders'}


for cat in security_category:
    if cat in categories_dict.keys():
        categories_dict[cat]['general'].add('general_house')
        categories_dict[cat]['main'].add('security_category')
    else:
        categories_dict[cat] = {'general': {'general_house'}, 'main': {'security_category'}}

# Cars

In [65]:
cars_gps_category = {'gps', 'portable vehicle gps', 'vehicle gps', 'in-dash navigation'}

for cat in cars_gps_category:
    if cat in categories_dict.keys():
        categories_dict[cat]['general'].add('general_cars')
        categories_dict[cat]['main'].add('cars_gps_category')
    else:
        categories_dict[cat] = {'general': {'general_cars'}, 'main': {'cars_gps_category'}}

In [66]:
cars_equipment_category = {'automotive - general', 'car stereos','audio car mounts', 
                        'car electronics', 'bluetooth car kits', 'car electronics accessories',
                        'car cradles', 'car chargers', 'car kits', 'car audio video', 'car stereos', 
                        'back seat cushions', 'vehicle mounts', 'dash mounting kits', 'mounts', 'cables adapters'}

for cat in cars_equipment_category:
    if cat in categories_dict.keys():
        categories_dict[cat]['general'].add('general_cars')
        categories_dict[cat]['main'].add('cars_equipment_category')
    else:
        categories_dict[cat] = {'general': {'general_cars'}, 'main': {'cars_equipment_category'}}

# Label Intents

In [68]:
equivalence_intent = {file: dict() for file in ['train', 'valid', 'test']}
brand_intent = {file: dict() for file in ['train', 'valid', 'test']}
main_cat_intent = {file: dict() for file in ['train', 'valid', 'test']}
general_cat_intent = {file: dict() for file in ['train', 'valid', 'test']}

In [69]:
file_names = ['train', 'valid', 'test']
files = [train0, valid0, test0]

for file, file_name in zip(files, file_names):
    for i, row in file.iterrows():
        if row['label'] == 1:
            equivalence_intent[file_name][i] = 1
            brand_intent[file_name][i] = 1
            main_cat_intent[file_name][i] = 1
            general_cat_intent[file_name][i] = 1
        else:
            equivalence_intent[file_name][i] = 0
            if df_a.iloc[row['ltable_id']]['brand'] == df_b.iloc[row['rtable_id']]['brand']:
                brand_intent[file_name][i] = 1
            else:
                brand_intent[file_name][i] = 0
                
            left_category = df_a.iloc[row['ltable_id']]['category']
            right_category = df_b.iloc[row['rtable_id']]['category']
            left_main_cats = categories_dict[left_category]['main']
            left_general_cats = categories_dict[left_category]['general']
            right_main_cats = categories_dict[right_category]['main']
            right_general_cats = categories_dict[right_category]['general']
            
            if left_main_cats.intersection(right_main_cats):
                main_cat_intent[file_name][i] = 1
            else:
                main_cat_intent[file_name][i] = 0
                
            if left_general_cats.intersection(right_general_cats):
                general_cat_intent[file_name][i] = 1
            else:
                general_cat_intent[file_name][i] = 0            
            

# Create Text Files

In [70]:
dfs = [train0, valid0, test0]
file_types = ['train', 'valid', 'test']
intents = [intent for intent in range(4)]
intent_dicts = [equivalence_intent, brand_intent, main_cat_intent, general_cat_intent]
for df, file_type in zip(dfs, file_types):
    for intent, intent_dict in zip(intents, intent_dicts):
        
        file1 = open("Walmart-Amazon/Walmart-Amazon_" + file_type +str(intent) + ".txt", "w", encoding="utf-8")
        for index, row in df.iterrows():
            left_title = df_a.iloc[row['ltable_id']]['title']
            right_title = df_b.iloc[row['rtable_id']]['title']
            left_category = df_a.iloc[row['ltable_id']]['category']
            right_category = df_b.iloc[row['rtable_id']]['category']
            left_modelno = df_a.iloc[row['ltable_id']]['modelno']
            right_modelno = df_b.iloc[row['rtable_id']]['modelno']
            left_price = df_a.iloc[row['ltable_id']]['price']
            right_price = df_b.iloc[row['rtable_id']]['price']
            
            left_title = " " if pd.isna(left_title) else left_title
            right_title = " " if pd.isna(right_title) else right_title
            left_modelno = " " if pd.isna(left_modelno) else left_modelno
            right_modelno = " " if pd.isna(right_modelno) else right_modelno
            left_price = " " if pd.isna(left_price) else left_price
            right_price = " " if pd.isna(right_price) else right_price
            
            text = "COL title VAL " + left_title  \
            + " COL modelno VAL " + str(left_modelno) \
            + " COL price VAL " + str(left_price) + "\t" + "COL title VAL " \
            + right_title + " COL modelno VAL " + str(right_modelno) + " COL price VAL " \
            + str(right_price) + "\t" + str(intent_dict[file_type][index]) + "\n"
            file1.write(text)
        file1.close()

## Multilabel

In [71]:
dfs = [train0, valid0, test0]
file_types = ['train', 'valid', 'test']
intents = [intent for intent in range(4)]
intent_dicts = [equivalence_intent, brand_intent, main_cat_intent, general_cat_intent]
for df, file_type in zip(dfs, file_types):
    file1 = open("Walmart-Amazon/Walmart-Amazon_" + file_type + "_Multilabel.txt", "w", encoding="utf-8")
    for index, row in df.iterrows():
        left_title = df_a.iloc[row['ltable_id']]['title']
        right_title = df_b.iloc[row['rtable_id']]['title']
        left_modelno = df_a.iloc[row['ltable_id']]['modelno']
        right_modelno = df_b.iloc[row['rtable_id']]['modelno']
        left_price = df_a.iloc[row['ltable_id']]['price']
        right_price = df_b.iloc[row['rtable_id']]['price']

        left_title = " " if pd.isna(left_title) else left_title
        right_title = " " if pd.isna(right_title) else right_title
        left_modelno = " " if pd.isna(left_modelno) else left_modelno
        right_modelno = " " if pd.isna(right_modelno) else right_modelno
        left_price = " " if pd.isna(left_price) else left_price
        right_price = " " if pd.isna(right_price) else right_price

        text = "COL title VAL " + left_title + " COL modelno VAL " + str(left_modelno) \
        + " COL price VAL " + str(left_price) + "\t" + "COL title VAL " \
        + right_title + " COL modelno VAL " + str(right_modelno) + " COL price VAL " \
        + str(right_price) + "\t" + str(equivalence_intent[file_type][index]) \
        + str(brand_intent[file_type][index]) + str(main_cat_intent[file_type][index]) \
        + str(general_cat_intent[file_type][index]) + "\n"
        file1.write(text)
    file1.close()

In [ ]:
intent_dicts = [equivalence_intent, brand_intent, main_cat_intent, general_cat_intent]